## Training test
### A notebook for prototyping mps training code


In [1]:
import numpy as np
import torch

from mps.models import MPS, ComplexTensor
from mps.utils import train_from_dict, rolling_avg, two_phase_training
import matplotlib.pyplot as plt
import datetime


Loaded libmkl_rt.so for dgesvd


Pick system size, dimensionality of local hilbert space, and initial bond dim

In [2]:
L=6
local_dim=2
bond_dim=2

In [3]:
# dev = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
dev = torch.device("cpu")

Initialize the MPS model

In [4]:
psi = MPS(L=L, local_dim=local_dim, bond_dim=bond_dim)

In [5]:
psi.to(device=dev)

In [6]:
fname_settings = "datasets/mps_sampled/ghz_plus_L=%d_angles.npy"%L
fname_samples = "datasets/mps_sampled/ghz_plus_L=%d_outcomes.npy"%L
fname_mps = "datasets/mps_sampled/ghz_plus_L=%d_state"%L
Nsamp=20000


In [7]:
training_settings=dict(batch_size=512,epochs=2,
                        max_sv=10,cutoff=1e-3,
                       lr_scale=1e-3, lr_timescale=100,
                       s2_scale=0, s2_timescale=100,
                       hold_early_cutoff=True,
                      mps_path = fname_mps, 
                       val_fraction=.2,
                      samples_per_epoch=5)
                           

In [ ]:
model, logdict1, logdict2, meta1, meta2 = two_phase_training(fname_samples, fname_settings, training_settings,
                                   N=Nsamp,
                                   use_cache=True, record_eigs=False,record_s2=True,verbose=True,
                                   compute_overlaps=False)

In [ ]:
fidelity = logdict['fidelity_mps']
loss = logdict['loss']
max_bond_dim = logdict['max_bond_dim']
smooth_loss = rolling_avg(loss,window=10*5)
# eigs = logdict['eigenvalues']
# s2 = logdict['s2']
# val_loss = logdict['val_loss']
# overlap = logdict['overlap']

In [ ]:
%matplotlib inline
fig, ax = plt.subplots()
plt.plot(loss, label='batch')
plt.plot(smooth_loss, label='smooth')
plt.legend()
plt.xlabel("training step")
plt.title("batch NLL loss")


In [ ]:
fig, ax = plt.subplots()
plt.plot(fidelity)

In [ ]:
fidelity[-1]